In [1]:

# import libraries
import requests
from PIL import Image

# suppress warnings
import warnings
warnings.filterwarnings("ignore")

try:
    import gradio as gr
    from together import Together

except ImportError:
    !pip install -q gradio
    !pip install -q together


import gradio as gr
from together import Together



In [2]:
dataset = {
    "input": [
        {
            "content": "Contract_V1.pdf",
            "category": "pdf",
            "source": "original_contract",
        },
        {
            "content": "Contract_V2.pdf",
            "category": "pdf",
            "source": "revised_contract",
        },
        {
            "content": "Technical_Report_V1.pdf",
            "category": "pdf",
            "source": "initial_technical_report",
        },
        {
            "content": "Technical_Report_V2.pdf",
            "category": "pdf",
            "source": "updated_technical_report",
        },
        {
            "content": "Blueprints_Scan.pdf",
            "category": "image_pdf",
            "source": "scanned_blueprint",
        },
        {
            "content": "Blueprints_Vector.pdf",
            "category": "vector_pdf",
            "source": "CAD_generated_blueprint",
        },
        {
            "content": "Electrical_Drawing_V1.pdf",
            "category": "vector_pdf",
            "source": "initial_electrical_drawing",
        },
        {
            "content": "Electrical_Drawing_V2.pdf",
            "category": "vector_pdf",
            "source": "updated_electrical_drawing",
        },
    ],
    "expected_output": [
    [
        {
            "question": "What changed in the payment terms of the revised contract?",
            "options": [
                "Late payments now incur a 5% penalty instead of 2%.",
                "Late payments are now waived.",
                "A new tax was introduced.",
                "The contract now requires prepayment."
            ],
            "correct": 0,
            "justification": "The revised contract increased the penalty for late payments from 2% to 5%."
        }
    ],
    [
        {
            "question": "What was updated in the technical report's efficiency analysis?",
            "options": [
                "Efficiency decreased to 80%.",
                "A new efficiency metric was added.",
                "Efficiency increased from 85% to 90%.",
                "The section was removed."
            ],
            "correct": 2,
            "justification": "The updated report reflects an increase in efficiency from 85% to 90%."
        }
    ],
    [
        {
            "question": "What structural change was detected in the blueprint?",
            "options": [
                "The number of support beams increased from 4 to 5.",
                "Support beams were removed.",
                "The building size was reduced.",
                "A new floor was added."
            ],
            "correct": 0,
            "justification": "The revised blueprint shows an increase in support beams from 4 to 5."
        }
    ],
    [
        {
            "question": "What was the main change in the CAD blueprint?",
            "options": [
                "The main column rotated from 90 to 85 degrees.",
                "The blueprint size was doubled.",
                "New rooms were added.",
                "The entire building was relocated."
            ],
            "correct": 0,
            "justification": "The revised blueprint shows that the main column structure was rotated from 90 to 85 degrees."
        }
    ],
    [
        {
            "question": "What change was detected in the electrical drawing?",
            "options": [
                "Transformer rating increased from 500 kVA to 600 kVA.",
                "Transformer size was reduced.",
                "A new transformer was removed.",
                "Transformer efficiency decreased."
            ],
            "correct": 0,
            "justification": "The revised electrical drawing shows an increase in transformer rating from 500 kVA to 600 kVA."
        }
    ]
],
}


In [3]:
import gradio as gr
import json
from together import Together


# TODO 1: Replace with your Together API key (https://www.together.ai/)
your_api_key = "9806a2601560024637df1e4acd804862faa67e08637db6598d920b64eebba43e"
client = Together(api_key=your_api_key)


def prompt_llm(prompt):
    # TODO 2: You can experiment with different models here (see here https://api.together.ai/models)
    model = "meta-llama/Meta-Llama-3-8B-Instruct-Lite"
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content

In [4]:
def show_data_point(index):
    input_data = dataset["input"][index]
    quiz_data = dataset["expected_output"][index]

    # Update header with current index + 1
    header_text = f"## Example {index + 1}"

    # Format the input display with markdown and symbols, now with a box
    input_text = """
<div style="border: 2px solid #ddd; border-radius: 8px; padding: 15px; background-color: #ddfff5;">

### 📖 Topic: {content}

🏷️ **Category**: {category}

🔗 **Source**: {source}

</div>
""".format(
        **input_data
    )

    # Format the output in markdown with styled box
    output_text = """
<div style="border: 2px solid #ddd; border-radius: 8px; padding: 15px; background-color: #ddfff5;">

### 📝 Quiz Questions

"""
    for i, quiz in enumerate(quiz_data, 1):
        output_text += f"#### Q{i}: {quiz['question']}\n\n"
        for j, option in enumerate(quiz["options"]):
            output_text += f"{chr(97 + j)}) {option}\n\n"
        output_text += f"\n✅ **Correct Answer**: {chr(96 + quiz['correct'] + 1)}\n"
        if "justification" in quiz:
            output_text += f"\n💡 **Explanation**: {quiz['justification']}\n"
        output_text += "\n---\n\n"

    output_text += "</div>"

    return header_text, input_text, output_text

In [7]:
import gradio as gr
import json
from datetime import datetime

# Define custom CSS for full dark mode with white borders
custom_css = """
/* Global dark mode */
body, .gradio-container {
    background-color: #1e1e1e !important;
    color: white !important;
}

/* Ensure all blocks, rows, and columns have dark backgrounds */
.gr-block, .gr-column, .gr-slider, .gr-button, .gr-row, .gr-box {
    background-color: #262626 !important;
    color: white !important;
    border-radius: 10px;
    padding: 10px;
    border: 2px solid white !important;
}

/* Fix for Markdown elements */
.gr-markdown, .gr-markdown h1, .gr-markdown h2, .gr-markdown h3, 
.gr-markdown h4, .gr-markdown p, .gr-markdown strong, 
.gr-markdown em, .gr-markdown a, .gr-markdown ul, 
.gr-markdown li, .gr-markdown pre, .gr-markdown code {
    color: white !important;
    background-color: transparent !important;
    opacity: 1 !important;  
}

/* Fix for input elements (slider, textbox, textarea) */
.gr-slider, .gr-textbox, .gr-textarea, input, textarea {
    background-color: #333 !important;
    color: white !important;
    border: 1px solid #555 !important;
}

/* Fix for Gradio UI components */
.gradio-container * {
    background-color: #1e1e1e !important;
    color: white !important;
}
"""

# Create Gradio interface
with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
    # 📚    PDFusion
    AI-Powered PDF Comparison.
    """
    )
    header = gr.Markdown("## Example 1")

    with gr.Row():
        index_slider = gr.Slider(
            minimum=0,
            maximum=len(dataset["input"]) - 1,
            step=1,
            value=0,
            label="Data Point Index",
            container=False,
        )

    with gr.Row():
        # Input section with prompt
        with gr.Column(scale=1):
            gr.Markdown("## Input")
            with gr.Row():
                input_text = gr.Markdown(
                    label="Input",
                    value=show_data_point(0)[1],
                )
        with gr.Column():
            run_prompt_btn = gr.Button("Run Prompt")

            prompt_llm_text = gr.TextArea(
                label="Prompt Template",
                value="""You are an expert quiz generator, skilled at creating engaging and educational multiple-choice questions.

# Task Description
Generate 3 multiple-choice questions about the given topic with 4
options each. Include explanations for the correct answers.

# Output Format
- Q1: Question 1
- A1: Option 1
- A2: Option 2
- A3: Option 3
- A4: Option 4
- Correct Answer: 1
- Explanation: Explanation for the correct answer

# Input Content
{content}

# Requirements
1. Each question must have exactly 4 options
2. Include clear explanations for correct answers
3. Ensure questions test understanding, not just memorization
4. Use clear, concise language
5. Make sure all options are plausible""",
                lines=10,
                interactive=True,
                container=True,
                show_label=True,
                elem_classes="custom-textarea",
            )

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("## Expected Output")
            output_text = gr.Markdown(
                label="Expected Output",
                value=show_data_point(0)[2],
            )

        # Output section with LLM output and expected output
        with gr.Column(scale=1):
            gr.Markdown("## LLM Output")
            llm_output = gr.TextArea(
                label="LLM Output",
                value="LLM output will appear here...",
                lines=10,
            )
            with gr.Row():
                approve_btn = gr.Button("Approve and Save")
                save_status = gr.Markdown("")

    # Function to save approved output
    def save_approved_output(index, prompt_template, llm_output):
        input_data = dataset["input"][index]
        output_data = {
            "input": input_data,
            "prompt": prompt_template,
            "llm_output": llm_output,
        }

        # Save to JSON file with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"approved_outputs_{timestamp}.json"

        with open(filename, "w") as f:
            json.dump(output_data, f, indent=4)

        return f"✅ Saved! File: `{filename}`"

    # Connect the approve button
    approve_btn.click(
        save_approved_output,
        inputs=[index_slider, prompt_llm_text, llm_output],
        outputs=[save_status],
    )

    index_slider.change(
        show_data_point,
        inputs=[index_slider],
        outputs=[header, input_text, output_text],
    )

    def generate_llm_response(index, prompt_template):
        input_data = dataset["input"][index]
        prompt = prompt_template.format(content=input_data["content"])
        response = prompt_llm(prompt)
        return response

    # Connect the run prompt button
    run_prompt_btn.click(
        generate_llm_response,
        inputs=[index_slider, prompt_llm_text],
        outputs=[llm_output],
    )

if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [9]:
import gradio as gr
import json
from together import Together

# ✅ API Setup
your_api_key = "9806a2601560024637df1e4acd804862faa67e08637db6598d920b64eebba43e"
client = Together(api_key=your_api_key)

# ✅ Function to Send Prompt to LLM
def prompt_llm(prompt):
    model = "meta-llama/Meta-Llama-3-8B-Instruct-Lite"
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content

# ✅ Function to Get Data Point
def show_data_point(index):
    input_data = dataset["input"][index]
    quiz_data = dataset["expected_output"][index]

    # Header update
    header_text = f"## Example {index + 1}"

    # Format input display
    input_text = """
<div style="border: 2px solid #ddd; border-radius: 8px; padding: 15px; background-color: #ddfff5;">
### 📖 Topic: {content}
🏷️ **Category**: {category}
🔗 **Source**: {source}
</div>
""".format(
        **input_data
    )

    # Format output
    output_text = """
<div style="border: 2px solid #ddd; border-radius: 8px; padding: 15px; background-color: #ddfff5;">
### 📝 Quiz Questions
"""
    for i, quiz in enumerate(quiz_data, 1):
        output_text += f"#### Q{i}: {quiz['question']}\n\n"
        for j, option in enumerate(quiz["options"]):
            output_text += f"{chr(97 + j)}) {option}\n\n"
        output_text += f"\n✅ **Correct Answer**: {chr(96 + quiz['correct'] + 1)}\n"
        if "justification" in quiz:
            output_text += f"\n💡 **Explanation**: {quiz['justification']}\n"
        output_text += "\n---\n\n"
    output_text += "</div>"

    return header_text, input_text, output_text

# ✅ Function to Generate LLM Response
def generate_llm_response(index, prompt_template):
    try:
        input_data = dataset["input"][index]
        prompt = prompt_template.format(content=input_data["content"])  # Format prompt

        print(f"🔍 Sending prompt to LLM:\n{prompt}")  # Debug log

        response = prompt_llm(prompt)  # Call the LLM

        if not response:  # Handle empty response
            return "⚠️ LLM did not return any output. Check API key or prompt format."

        print(f"✅ Received LLM response:\n{response}")  # Debug log

        return response  # Return the generated response

    except Exception as e:
        print(f"❌ Error while generating LLM response: {e}")  # Debugging error
        return f"⚠️ Error: {e}"

# ✅ Create Gradio Interface






import gradio as gr
import json
from together import Together

# ✅ Custom CSS for Dark Mode (Place This at the Top)
custom_css = """
/* Global dark mode */
body, .gradio-container {
    background-color: #1e1e1e !important;
    color: white !important;
}

/* Ensure all blocks, rows, and columns have dark backgrounds */
.gr-block, .gr-column, .gr-slider, .gr-button, .gr-row, .gr-box {
    background-color: #262626 !important;
    color: white !important;
    border-radius: 10px;
    padding: 10px;
    border: 2px solid white !important;
}

/* Fix for Markdown elements */
.gr-markdown, .gr-markdown h1, .gr-markdown h2, .gr-markdown h3, 
.gr-markdown h4, .gr-markdown p, .gr-markdown strong, 
.gr-markdown em, .gr-markdown a, .gr-markdown ul, 
.gr-markdown li, .gr-markdown pre, .gr-markdown code {
    color: white !important;
    background-color: transparent !important;
    opacity: 1 !important;  
}

/* Fix for input elements (slider, textbox, textarea) */
.gr-slider, .gr-textbox, .gr-textarea, input, textarea {
    background-color: #333 !important;
    color: white !important;
    border: 1px solid #555 !important;
}

/* Fix for Gradio UI components */
.gradio-container * {
    background-color: #1e1e1e !important;
    color: white !important;
}
"""

# ✅ Apply custom CSS inside gr.Blocks(...)
with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
    # 📚 Omniscient Prompt XRay
    Explore quiz questions generated for different topics.
    """
    )
    header = gr.Markdown("## Example 1")

    with gr.Row():
        index_slider = gr.Slider(
            minimum=0,
            maximum=len(dataset["input"]) - 1,
            step=1,
            value=0,
            label="Data Point Index",
            container=False,
        )

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("## Input")
            with gr.Row():
                input_text = gr.Markdown(
                    label="Input",
                    value=show_data_point(0)[1],
                )

        with gr.Column():
            run_prompt_btn = gr.Button("Run Prompt")

            prompt_llm_text = gr.TextArea(
                label="Prompt Template",
                value="""You are an expert quiz generator, skilled at creating engaging and educational multiple-choice questions.

# Task Description
Generate 3 multiple-choice questions about the given topic with 4
options each. Include explanations for the correct answers.

# Output Format
- Q1: Question 1
- A1: Option 1
- A2: Option 2
- A3: Option 3
- A4: Option 4
- Correct Answer: 1
- Explanation: Explanation for the correct answer

# Input Content
{content}

# Requirements
1. Each question must have exactly 4 options
2. Include clear explanations for correct answers
3. Ensure questions test understanding, not just memorization
4. Use clear, concise language
5. Make sure all options are plausible""",
                lines=10,
                interactive=True,
                container=True,
                show_label=True,
                elem_classes="custom-textarea",
            )

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("## Expected Output")
            output_text = gr.Markdown(
                label="Expected Output",
                value=show_data_point(0)[2],
            )

        with gr.Column(scale=1):
            gr.Markdown("## LLM Output")
            llm_output = gr.TextArea(
                label="LLM Output",
                value="LLM output will appear here...",
                lines=10,
            )
            with gr.Row():
                approve_btn = gr.Button("Approve and Save")
                save_status = gr.Markdown("")

    # ✅ Connect LLM Generation to Button
    run_prompt_btn.click(
        generate_llm_response,  # Calls function
        inputs=[index_slider, prompt_llm_text],  # Uses index & prompt template
        outputs=[llm_output],  # Updates the LLM output field
    )

    # ✅ Update Interface When Slider Changes
    index_slider.change(
        show_data_point,
        inputs=[index_slider],
        outputs=[header, input_text, output_text],
    )

if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\MuriloFarias\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\MuriloFarias\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\MuriloFarias\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gradio\blocks.py", line 2045, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\MuriloFarias\AppData\Local\Packages\PythonSoftwareFoundation

In [10]:
import gradio as gr
import json
from together import Together

# ✅ Set up Together AI API
your_api_key = "9806a2601560024637df1e4acd804862faa67e08637db6598d920b64eebba43e"
client = Together(api_key=your_api_key)

# ✅ Custom CSS for Full Dark Mode
custom_css = """
/* Global dark mode */
body, .gradio-container {
    background-color: #1e1e1e !important;
    color: white !important;
}

/* Ensure all blocks, rows, and columns have dark backgrounds */
.gr-block, .gr-column, .gr-slider, .gr-button, .gr-row, .gr-box {
    background-color: #262626 !important;
    color: white !important;
    border-radius: 10px;
    padding: 10px;
    border: 2px solid white !important;
}

/* Fix for Markdown elements */
.gr-markdown, .gr-markdown h1, .gr-markdown h2, .gr-markdown h3, 
.gr-markdown h4, .gr-markdown p, .gr-markdown strong, 
.gr-markdown em, .gr-markdown a, .gr-markdown ul, 
.gr-markdown li, .gr-markdown pre, .gr-markdown code {
    color: white !important;
    background-color: transparent !important;
    opacity: 1 !important;  
}

/* Fix for input elements (slider, textbox, textarea) */
.gr-slider, .gr-textbox, .gr-textarea, input, textarea {
    background-color: #333 !important;
    color: white !important;
    border: 1px solid #555 !important;
}

/* Fix for Gradio UI components */
.gradio-container * {
    background-color: #1e1e1e !important;
    color: white !important;
}
"""

# ✅ Function to Call LLM API
def prompt_llm(prompt):
    try:
        model = "meta-llama/Meta-Llama-3-8B-Instruct-Lite"
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
        )
        return response.choices[0].message.content  # Return the LLM-generated response

    except Exception as e:
        print(f"❌ API Error: {e}")  # Debugging
        return f"⚠️ Error: {e}"

# ✅ Function to Display a Data Point
def show_data_point(index):
    input_data = dataset["input"][index]
    quiz_data = dataset["expected_output"][index]

    # Header update
    header_text = f"## Example {index + 1}"

    # Format input display
    input_text = """
<div style="border: 2px solid white; border-radius: 8px; padding: 15px; background-color: #262626;">
### 📖 Topic: {content}
🏷️ **Category**: {category}
🔗 **Source**: {source}
</div>
""".format(
        **input_data
    )

    # Format output
    output_text = """
<div style="border: 2px solid white; border-radius: 8px; padding: 15px; background-color: #262626;">
### 📝 Quiz Questions
"""
    for i, quiz in enumerate(quiz_data, 1):
        output_text += f"#### Q{i}: {quiz['question']}\n\n"
        for j, option in enumerate(quiz["options"]):
            output_text += f"{chr(97 + j)}) {option}\n\n"
        output_text += f"\n✅ **Correct Answer**: {chr(96 + quiz['correct'] + 1)}\n"
        if "justification" in quiz:
            output_text += f"\n💡 **Explanation**: {quiz['justification']}\n"
        output_text += "\n---\n\n"
    output_text += "</div>"

    return header_text, input_text, output_text

# ✅ Function to Generate LLM Response
def generate_llm_response(index, prompt_template):
    try:
        input_data = dataset["input"][index]  # Get the input data for selected index
        prompt = prompt_template.format(content=input_data["content"])  # Format prompt

        print(f"🔍 Sending prompt to LLM:\n{prompt}")  # Debug log

        response = prompt_llm(prompt)  # Call the LLM API

        if not response or response.strip() == "":
            print("⚠️ LLM returned an empty response.")
            return "⚠️ LLM did not return any output. Check API key or prompt format."

        print(f"✅ Received LLM response:\n{response}")  # Debug log

        return response  # Return the generated response

    except Exception as e:
        print(f"❌ Error while generating LLM response: {e}")  # Debugging error
        return f"⚠️ Error: {e}"

# ✅ Create Gradio Interface
with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 📚 Omniscient Prompt XRay\nExplore quiz questions generated for different topics.")

    header = gr.Markdown("## Example 1")

    with gr.Row():
        index_slider = gr.Slider(
            minimum=0,
            maximum=len(dataset["input"]) - 1,
            step=1,
            value=0,
            label="Data Point Index",
            container=False,
        )

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("## Input")
            with gr.Row():
                input_text = gr.Markdown(
                    label="Input",
                    value=show_data_point(0)[1],
                )

        with gr.Column():
            run_prompt_btn = gr.Button("Run Prompt")

            prompt_llm_text = gr.TextArea(
                label="Prompt Template",
                value="""You are an expert quiz generator, skilled at creating engaging and educational multiple-choice questions.

# Task Description
Generate 3 multiple-choice questions about the given topic with 4
options each. Include explanations for the correct answers.

# Input Content
{content}

# Requirements
1. Each question must have exactly 4 options
2. Include clear explanations for correct answers
3. Ensure questions test understanding, not just memorization
4. Use clear, concise language
5. Make sure all options are plausible""",
                lines=10,
                interactive=True,
                container=True,
                show_label=True,
                elem_classes="custom-textarea",
            )

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("## Expected Output")
            output_text = gr.Markdown(
                label="Expected Output",
                value=show_data_point(0)[2],
            )

        with gr.Column(scale=1):
            gr.Markdown("## LLM Output")
            llm_output = gr.TextArea(
                label="LLM Output",
                value="LLM output will appear here...",
                lines=10,
            )
            with gr.Row():
                approve_btn = gr.Button("Approve and Save")
                save_status = gr.Markdown("")

    # ✅ Connect LLM Generation to Button
    run_prompt_btn.click(
        generate_llm_response,  # Calls function
        inputs=[index_slider, prompt_llm_text],  # Uses index & prompt template
        outputs=[llm_output],  # Updates the LLM output field
    )

    # ✅ Update Interface When Slider Changes
    index_slider.change(
        show_data_point,
        inputs=[index_slider],
        outputs=[header, input_text, output_text],
    )

if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\MuriloFarias\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\MuriloFarias\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\MuriloFarias\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gradio\blocks.py", line 2045, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\MuriloFarias\AppData\Local\Packages\PythonSoftwareFoundation